In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, *-*"

Installed Packages Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.21606.1

In [ ]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Collections;
using System.Threading;

In [ ]:
var input = await File.ReadAllLinesAsync("input.txt");

In [ ]:
#nullable enable

public record Range{
    public long Min {get; init;}
    public long Max {get; init;}

    public Range? Overlap(Range other)
    {
        if(Max >= other.Min && Min <= other.Max)
        {
            return new Range{
                Min = Math.Max(Min, other.Min),
                Max = Math.Min(Max, other.Max)
            };
        }

        return null;
    }

    public long Count()
    {
        return Max - Min + 1;
    }

    public IEnumerable<int> Explode()
    {
        return Enumerable.Range((int) Min, (int) Count());
    }
}

In [ ]:
#nullable enable

public record Box
{
    public Range X {get; init;}
    public Range Y {get; init;}
    public int Light {get; set;} = 1;

    public Box(){}

    public Box(IEnumerable<string> input)
    {
        var numbers = input.Select(c => int.Parse(c)).ToArray();
        X = new Range{Min = numbers[0], Max = numbers[2]};
        Y = new Range{Min = numbers[1], Max = numbers[3]};
    }

    public bool Overlaps(Box other)
    {
        if(X.Overlap(other.X) == null)
        {
            return false;
        }

        if(Y.Overlap(other.Y) == null)
        {
            return false;
        }

        return true;
    }

    public Box? Overlap(Box other, int lightDiff = 0)
    {
        var x = X.Overlap(other.X);
        var y = Y.Overlap(other.Y);

        if(x is not null && y is not null)
        {
            return new Box{
                X = x,
                Y = y,
                Light = Light + lightDiff
            };
        }

        return null;
    }

    public long Count()
    {
        return X.Count() * Y.Count() * Light;
    }

    private static int Color = 0;

    public static int GetColor()
    {
        return Color++;
    }

    public IEnumerable<(int x, int y, int size, int color)> Explode()
    {
        var color = GetColor();

        foreach(var x in X.Explode())
        {
            foreach(var y in Y.Explode())
            {
                yield return (x, y, 3, color);                
            }
        }
    }

    public IEnumerable<Box> Subtract(Box other, int lightDiff = 0)
    {
        var overlap = Overlap(other);

        if(overlap is null)
        {
            yield break;
        }

        if(X.Min < overlap.X.Min)
        {
            yield return this with
            {
                X = X with {Max = overlap.X.Min - 1},
                Light = Light + lightDiff
            };
        }

        if(X.Max > overlap.X.Max)
        {
            yield return this with
            {
                X = X with {Min = overlap.X.Max + 1},
                Light = Light + lightDiff
            };
        }

        if(Y.Min < overlap.Y.Min)
        {
            yield return this with
            {
                X = overlap.X,
                Y = Y with {Max = overlap.Y.Min - 1},
                Light = Light + lightDiff
            };
        }

        if(Y.Max > overlap.Y.Max)
        {
            yield return this with
            {
                X = overlap.X,
                Y = Y with {Min = overlap.Y.Max + 1},
                Light = Light + lightDiff
            };
        }
    }
}

In [ ]:
public static void Subtract(List<Box> boxes, Box right)
{    
    var overlaps = boxes.Where(c => c.Overlaps(right)).ToArray();

    if(!overlaps.Any())
    {                
        return;
    }

    foreach(var overlap in overlaps)
    {
        boxes.Remove(overlap);
        boxes.AddRange(overlap.Subtract(right));
    }
}

In [ ]:
public static void Add(List<Box> boxes, Box right)
{
    var overlaps = boxes.Where(c => c.Overlaps(right)).ToArray();

    if(!overlaps.Any())
    {
        boxes.Add(right);
        return;
    }

    var total = new List<Box>() {right};

    foreach(var overlap in overlaps)
    {
        if(!total.Any())
        {
            break;
        }
        
        var subtotal = new List<Box>();
        foreach(var remaining in total)
        {
            if(remaining.Overlaps(overlap))
            {                
                subtotal.AddRange(remaining.Subtract(overlap));
            }
            else
            {                    
                subtotal.Add(remaining);
            }                    
        }

        total = subtotal;
    }

    boxes.AddRange(total);
}

In [ ]:
public static void Toggle(List<Box> boxes, Box right)
{
    var overlaps = boxes.Where(c => c.Overlaps(right)).ToArray();

    if(!overlaps.Any())
    {
        boxes.Add(right);
        return;
    }

    var total = new List<Box>() {right};

    foreach(var overlap in overlaps)
    {
        boxes.AddRange(overlap.Subtract(right));
        boxes.Remove(overlap);

        if(!total.Any())
        {
            break;
        }
        
        var subtotal = new List<Box>();
        foreach(var remaining in total)
        {
            if(remaining.Overlaps(overlap))
            {
                subtotal.AddRange(remaining.Subtract(overlap));                
            }
            else
            {
                subtotal.Add(remaining);
            }                    
        }

        total = subtotal;
    }

    boxes.AddRange(total);
}

In [ ]:
public class ChristmasLights
{
    List<Box> Boxes {get;} = new List<Box>();

    public ChristmasLights(IEnumerable<string> input)
    {
        foreach(var line in input)
        {
            var segments = line.Split(new [] {"through", ",", " "}, 6, StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries);
            if(segments[0] == "toggle")
            {
                Toggle(Boxes, new Box(segments.Skip(1)));
                continue;
            }

            if(segments[1] == "on")
            {
                Add(Boxes, new Box(segments.Skip(2)));
                continue;
            }

            Subtract(Boxes, new Box(segments.Skip(2)));
        }
    }

    public long Count()
    {
        if(!Boxes.Any())
        {
            return 0;
        }

        return Boxes.Select(c => c.Count()).Aggregate((a, b) => a + b);
    }
}

In [ ]:
// var lights = new ChristmasLights(input);
// lights.Count()

In [ ]:
public class ShitLight
{
    public int[,] Map = new int[1000,1000];

    public ShitLight(IEnumerable<string> input)
    {
        foreach(var line in input)
        {
            var segments = line.Split(new [] {"through", ",", " "}, 6, StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries);
            
            
            if(segments[0] == "toggle")
            {
                var (x, y) = Parse(segments.Skip(1));
                A(x, y, 2);
                continue;
            }

            var (x, y) = Parse(segments.Skip(2));
            if(segments[1] == "on")
            {
                A(x, y);
                continue;
            }

            S(x, y);
        }
    }

    public (Range x, Range x) Parse(IEnumerable<string> input)
    {
        var numbers = input.Select(c => int.Parse(c)).ToArray();
        return (
            new Range{Min = numbers[0], Max = numbers[2]},
            new Range{Min = numbers[1], Max = numbers[3]}
            );
    }

    public void A(Range x, Range y, int diff = 1)
    {
        var numbers = input.Select(c => int.Parse(c)).ToArray();
        X = new Range{Min = numbers[0], Max = numbers[2]};
        Y = new Range{Min = numbers[1], Max = numbers[3]};
    }

    public void S(Range x, Range y)
    {
        var numbers = input.Select(c => int.Parse(c)).ToArray();
        X = new Range{Min = numbers[0], Max = numbers[2]};
        Y = new Range{Min = numbers[1], Max = numbers[3]};
    }
}

Error: (38,9): error CS0103: The name 'X' does not exist in the current context
(39,9): error CS0103: The name 'Y' does not exist in the current context
(45,9): error CS0103: The name 'X' does not exist in the current context
(46,9): error CS0103: The name 'Y' does not exist in the current context